# TODO
* add examples
* add basic unit tests for the given input data
* test the actual excel modification code
* issues:
    * column index consistency seems to be off
    * label highlighting is not consistent
* maybe do a second pass asking if the relevant data is highlighted
* if first pass does not highlight any data, do a 2 pass approach where you generate potential synonyms or relevant pieces of data
* try a multi model approach, sending easier queries to gpt-3.5 etc

In [13]:
! pip install --upgrade pip
! pip install openai
! pip install langchain
! pip install langchain_openai
! pip install openpyxl
! pip install pandas

  Using cached tiktoken-0.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached regex-2023.12.25-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached tiktoken-0.5.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached regex-2023.12.25-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (789 kB)


In [1]:
import openpyxl
import openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import json
from langchain_core.utils.function_calling import convert_to_openai_tool

/tmp/ipykernel_4856/3259953313.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
wb = openpyxl.load_workbook(filename='data/sample_input.xlsx')

In [3]:
wb.sheetnames

["NOW-US, IS FY'23", "NOW-US, BS FY'23", "NOW-US, CF FY'23"]

In [4]:
wb.active

<Worksheet "NOW-US, CF FY'23">

In [5]:
ws = wb.active

In [6]:
ws.max_column, ws.max_row

(4, 61)

In [7]:
df = pd.read_excel('data/sample_input.xlsx')
df

,FactSet Filings Wizard,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,SERVICENOW INC,NaN,NaN,NaN
1,NOW-US,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,Annual Report - FY 2023 - Income Statement,NaN,NaN,NaN
4,Open in Workstation,NaN,NaN,NaN
5,Open in Web,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,Consolidated Statements of Comprehensive Incom...,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,"12 Months Ended Dec. 31, 2023","12 Months Ended Dec. 31, 2022","12 Months Ended Dec. 31, 2021"


In [36]:
res_str = '|' + '|'.join(str(i) for i in range(ws.max_column+1)) + '|\n' 
res_str += '|' + '|'.join('---' for i in range(ws.max_column+1)) + '|\n'
for row in range(1, ws.max_row+1):
    res_str += f'|{row}|' + '|'.join(str(ws.cell(row=row, column=col).value).replace('\n', '') if ws.cell(row=row, column=col).value else '' for col in range(1, ws.max_column+1)) + '|\n'
res_str.split('\n')

['|0|1|2|3|4|',
 '|---|---|---|---|---|',
 '|1|FactSet Filings Wizard||||',
 '|2|SERVICENOW INC||||',
 '|3|NOW-US||||',
 '|4|||||',
 '|5|Annual Report - FY 2023 - Cash Flow Statement||||',
 '|6|Open in Workstation||||',
 '|7|Open in Web||||',
 '|8|||||',
 '|9|Consolidated Statements of Cash Flows - USD ($) $ in Millions||||',
 '|10|||||',
 '|11||12 Months Ended Dec. 31, 2023|12 Months Ended Dec. 31, 2022|12 Months Ended Dec. 31, 2021|',
 '|12|||||',
 '|13|Cash flows from operating activities:||||',
 '|14|Net income|1731|325|230|',
 '|15|||||',
 '|16|Adjustments to reconcile net income to net cash provided by operating activities:||||',
 '|17|Depreciation and amortization|562|433|472|',
 '|18|Amortization of deferred commissions|459|358|294|',
 '|19|Stock-based compensation|1604|1401|1131|',
 '|20|Deferred income taxes|-857|15|-34|',
 '|21|Other||17|40|',
 '|22|||||',
 '|23|Changes in operating assets and liabilities, net of effect of business combinations:||||',
 '|24|Accounts receivab

In [152]:
def highlight_cells(cells: list[tuple[int, int]]):
    """Highlight the given cells as containing data of interest.

    Args:
        cells: array of tuples of integers, where each element corresponds to a cell: the first element of the tuple corresponds to the row and the second integer corresponds to the column of the cell. Formatted like this example: `[(5,1), (5,2), (7,1), (7,2)]` This argument **CANNOT** be an empty array.
    """
    pass

json.dumps(convert_to_openai_tool(highlight_cells))

'{"type": "function", "function": {"name": "highlight_cells", "description": "Highlight the given cells as containing data of interest.", "parameters": {"type": "object", "properties": {"cells": {"description": "array of tuples of integers, where each element corresponds to a cell: the first element of the tuple corresponds to the row and the second integer corresponds to the column of the cell. Formatted like this example: `[(5,1), (5,2), (7,1), (7,2)]` This argument **CANNOT** be an empty array."}}, "required": ["cells"]}}}'

In [156]:
llm = ChatOpenAI(model_name='gpt-4').bind_tools([highlight_cells])
system_prompt = """You are a financial advisor helping users find information in tables. You will be given a financial statement table and a particular piece of information to find in that table. You can use the highlight_cells tool to help the user find the relevant information in the table.
## General Guidelines
* The table might be formatted in different ways, but there will likely be labels for kinds of data on a header row and column, and corresponding data points further on that row or column.
* Your task is to highlight all cells that contain the kind of data that the user is looking for, using the provided highlight_cells tool.
* The data with the exact title user asked for might not be present. In those cases, highlight any cell that might be relevant or that might help derive the target data.
* Assume that this is the only document the user has, and do your best to help them get as close as they can to the information they need.
* If multiple cells contain the kind of data the user is asking for (for example the revenue data for different years or quarters), highlight all of them.
* Be sure to highlight **ALL** the individual cells that contain the data points, across any time scale (unless a specific time range is specified by the user).
* You cannot chat with the user directly, and you cannot ask them whether they want you to do something different. When in doubt, highlight the most relevant data you can find."""
human_template = "I'm looking for {data_title} in this table:\n{table_string}"
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', human_template),
])
chain = prompt | llm

In [157]:
data_titles = [
    'Revenue',
    'Cost of Goods Sold (COGS)',
    'Operating Expenses',
    'Other Income/Expenses',
    'Interest Expense',
    'Depreciation and Amortization',
    'Stock-Based Compensation',
    'Capital Expenditures',
    'Gain/Loss on Assets',
]

In [158]:
responses = {title: chain.invoke({'data_title': title, 'table_string': res_str}) for title in data_titles}

In [159]:
def print_res(res):
    print(res.content)
    print(res.additional_kwargs)
    print('\n')

In [160]:
for title, res in responses.items():
    print(title)
    print_res(res)

Revenue

{'tool_calls': [{'id': 'call_O3ZzCyV4dHT0faM4eDV6otkK', 'function': {'arguments': '{\n  "cells": []\n}', 'name': 'highlight_cells'}, 'type': 'function'}]}


Cost of Goods Sold (COGS)
I'm sorry, but the table you've provided is a cash flow statement and does not contain information on Cost of Goods Sold (COGS). COGS is typically found in an income statement. Please provide an income statement to locate the COGS.
{}


Operating Expenses
I'm sorry, but the table you provided is a Cash Flow Statement and does not contain information about Operating Expenses. The Operating Expenses can usually be found in an Income Statement. Would you like to provide an Income Statement or look for some other information in the Cash Flow Statement instead?
{}


Other Income/Expenses

{'tool_calls': [{'id': 'call_mrN3z5vbxxJpo8ltEK9svq5s', 'function': {'arguments': '{\n  "cells": [(21, 2), (21, 3), (21, 4)]\n}', 'name': 'highlight_cells'}, 'type': 'function'}]}


Interest Expense

{'tool_calls': [{